## Converting csv files into .txt to Compare two object detection models

If the bounding box is provided in a csv format, we can convert it into .txt and for comparing the two models vesus ground truth, we can apply https://github.com/rafaelpadilla/Object-Detection-Metrics

In [ ]:
# Import all the required libraries
import os 
import glob 
import argparse
import pandas as pd
from pandas.core.algorithms import unique

In [ ]:
# Import object_detections dataset
path = 'data/gt_boxes.csv'
save_text_path = 'data/GT/'

path1 = 'data/model_1.csv'
save_text_path1 = 'data//model_1/'

path2 = 'data/model_2.csv'
save_text_path2 = 'data/model_2/'

In [ ]:
# converting csv files into .txt 
def convert_coordinates(size, box):
    dw = 1.0/size[0]
    dh = 1.0/size[1]
    x = (box[0]+box[1])/2.0
    y = (box[2]+box[3])/2.0
    w = box[1]-box[0]
    h = box[3]-box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

def csv_to_txt(csv_path ,out_path):
    df = pd.read_csv(csv_path) # Read Csv 
    unique_classes = pd.unique(df['label']) # Create label map
    labels = {}
    for i,label in enumerate(unique_classes):
        labels[label]=i
    # Group rows based on filename bcz single file may have multiple annotations
    for name, group in df.groupby('image_id'): #for name, group in df.groupby('filename'):
        # Create filename
        fname_out = os.path.join( out_path, str(name) + '.txt')
        # Open txt file to write
        with open(fname_out, "w") as f:
            # Iter through each bbox
            for row_index, row in group.iterrows():
                xmin = row['xmin']
                ymin = row['ymin']
                xmax = row['xmax']
                ymax = row['ymax']
                width = 200
                height = 300
                label = row['label']  
                score = row['score']  
                # Get label index
                label_str = str(labels[label])
                b = (float(score), float(xmin), float(xmax), float(ymin), float(ymax))
                # Convert bbox from pascal voc format to yolo txt format
                bb = convert_coordinates((width,height), b)
                # Write into file
                f.write(label_str + " " + " ".join([("%.6f" % a) for a in b]) + '\n')

if __name__ == '__main__':
    csv_to_txt(path , save_text_path) # ground truth
    csv_to_txt(path1 , save_text_path1) # model 1
    csv_to_txt(path2 , save_text_path2) # model 2